# P3
## Desenvolvido e discutido por Felipe Arruda-196862 e Leonardo Koike-

Evolucionary Model:
* The evolutionary model adopted -> Genetic Algorithms
* Variations on parameters -> Population size/Crossover rate/Mutation rate/Generation interval/Stopping criteria
* Fitness function adopted -> combination bwtween max score and has won
* Population size -> population_size
* Stop criteria -> max_generation or population_average hasn`t changed
* Selection technique -> Roulette Method
* Crossover technique -> Single Point Crossover
* Replacement method -> Steady State, bring 10% of the original population among with the children
* Mutation rate -> 0.2
* Crossover rate -> 0.8

Decidido por fixar o caminho dos fantamas, pois gerava um problema que conseguiriamos resolver com o individuo que criamos

In [32]:
## Imports
import random
import numpy as np
from statistics import mean
random.seed(10)
maps = ['smallClassic', 'mediumClassic', 'originalClassic']

In [33]:
## Inicialização da população com tamanho passado pelo parâmetro
# retorna a população criada
def population_inicialization(population_size, min_carac, max_carac):
    population = []
    for i in range(population_size):
        len  = random.randint(min_carac, max_carac)
        individuo = ""
        for c in range(len):
            carac = random.randint(1, 100)
            if carac >= 50:
                individuo = individuo + 'L'
            else:
                individuo = individuo + 'R'
        population.append(individuo)
    population = np.array(population)
    return population

In [34]:
# Função para realização de crossover de 1 ponto entre 2 individuos
# retorna os filhos destes individuos
def crossover(l, q):
    # converting the string to list for performing the crossover
    l = list(l)
    q = list(q)
  
# generating the random number to perform crossover
    size_l = len(l)
    size_q = len(q)
    size = 0
    size = min(size_l, size_q)
    k = random.randint(0, size-1)
  
# interchanging the genes
    for i in range(0, k):
        l[i], q[i] = q[i], l[i]
    
    l = ''.join(l)
    q = ''.join(q)
    return l, q

In [35]:
# Função que muta 1 gene de um indivíduo
# Retorna individuo mutado
def mutation(i):
    i = list(i)
    
    x = random.randint(0, len(i)-1)
    if i[x] == 'L':
        i[x] = 'R'
    else:
        i[x] = 'L'
    
    i = ''.join(i)
    return i

In [36]:
#selects 1 or 2 individuals in the population taking into account their fitness
def selection(ind_amount, fitness):
    ind = []

    #section to prevent negative values
    min_val = np.min(fitness)
    if min_val < 0:
        fitness = fitness - min_val

    #Exponential Transformation for the fitness
    fitness = np.sqrt(fitness + 1)

    #Selection
    max_val = np.sum(fitness)
    for n in range(ind_amount):
        pick = random.uniform(0, max_val)
        current = 0
        for i in range(len(fitness)):
            current += fitness[i]
            if current > pick:
                ind.append(i)
                break
    return ind

In [37]:
# Função para mudar a geração, recebe a população pai e gera os filhos
# Retorna uma nova população
def change_generation(population, stedy_rate, mutation_rate, crossover_rate, fitness):
    new_pop = []
    fit_temp = fitness.copy()
    pop_temp = population.copy()

    # stedy state part
    print()
    for i in range(int(stedy_rate*len(population))):
        if population[np.argmax(fitness)] not in new_pop:
            new_pop.append(population[np.argmax(fitness)])
        else: # esse if impede que eu fique repetindo o mesmo individuo muitas vezes pela seleção dos n melhores da população
            i = i - 1 
        fitness = np.delete(fitness, np.argmax(fitness))
        population = np.delete(population, np.argmax(fitness))
        
    
    population = pop_temp
    fitness = fit_temp

    #crossover part
    for i in range(int(crossover_rate*len(population)/2)):
        idx = selection(2, fitness)
        a, b = crossover(population[idx[0]], population[idx[1]])
        new_pop.append(a)
        new_pop.append(b)
    
    #mutation part
    for i in range(int(mutation_rate*len(population))):
        idx = selection(1, fitness)
        new_pop.append(mutation(population[idx[0]]))
    
    new_pop = np.array(new_pop)
    return new_pop

In [38]:
# Função para gerar o fitness da população
# ela joga o jogo com cada indivíduo e gera as métricas
# retorna as métricas dessa população
def playGames(population, maps):
    #initializate generation calculations
    population_avgs = []
    population_wins = []

    for i in population:
        i_scores = []
        i_win = []
        for mapa in maps:
            result = !python pacman.py -p EvolutionAgent -a steps=$i -l $mapa -q -f
            is_win = int(result[3].split(':')[1].split('/')[0].replace(" ", ""))
            i_win.append(is_win)
            score = float(result[1].split(':')[1].replace(" ", ""))
            if is_win == 1 and mapa == 'smallClassic':
                score = score + 150
            elif is_win == 1 and mapa == 'mediumClassic':
                score = score + 500
            elif is_win == 1 and mapa == 'originalClassic':
                score = score + 1200
            i_scores.append(score)

        ind_mean = np.mean(i_scores)
        population_avgs = population_avgs + [ind_mean]
        wins = np.sum(i_win)
        population_wins.append(wins)
        
    population_avgs = np.array(population_avgs)
    population_wins = np.array(population_wins)
    return population_avgs, population_wins

In [39]:
# Função para retornar se o treino ainda deve começar se a média não mudar muito pouco
def checkStopCriteria_avg(current_gen, prev_gen):
    diff = current_gen - prev_gen
    ratio = diff/current_gen
    if ratio < 0:
        ratio = ratio * -1
    if ratio < 0.005:
        return True
    return False

In [40]:
def checkStopCriteria_win(current_gen, prev_gen):
    status = False
    if current_gen >= prev_gen:
        status = True
    return status

In [41]:
# Função para treino da população
def treino(population, maps, max_generation, stedy_rate, mutation_rate, crossover_rate):
    
    #vetor com as populações
    generations =[]
    generations.append(population)

    # valores para avaliação da população
    generation_avg = []
    generation_max_value = []
    generation_max_ind = []
    generation_min_value = []
    generation_win_rate = []

    # variavel que irá controlar o critério de parada do treino
    stop = False

    for g in range(max_generation):
        print("Generation " + str(g))

        # process the fitness for each individual in population
        avgs, wins = playGames(generations[g], maps)

        #calculate the genaration metrics
        generation_max_value.append(np.amax(avgs))
        max_ind_idx = np.where(avgs == generation_max_value[g])
        generation_max_ind.append(generations[g][max_ind_idx[0]])
        generation_min_value.append(np.amin(avgs))
        generation_avg.append(np.mean(avgs))
        total_wins = np.sum(wins)
        total_games = (len(maps)*population.shape[0])
        generation_win_rate.append(total_wins/total_games)
        

        print("Avg = {} \nWin rate = {}\nMax Value = {}\nMin value = {}".format(generation_avg[g], generation_win_rate[g], generation_max_value[g], generation_min_value[g]))
        print("max_ind = {}".format(max_ind_idx))

        

        #Controle para critério de parada do treino
        if g > 0:
            stop = checkStopCriteria_avg(generation_avg[g], generation_avg[g-1]) and checkStopCriteria_win(generation_win_rate[g], generation_win_rate[g-1])
        if stop:
            break

        # se continuar o treino, eu gero a próxima população
        # With the fitness, generate the next population
        next_gen = change_generation(generations[g], stedy_rate, mutation_rate, crossover_rate, avgs)
        generations.append(next_gen)
    return generations, generation_avg, generation_max_value, generation_min_value, generation_win_rate, generation_max_ind


In [42]:
## Parâmetros do primeiro modelo
population_size = 100
max_generation = 1000
stedy_rate = 0.1
mutation_rate = 0.1
crossover_rate = 0.8
min_carac = 4
max_carac = 50
population_1 = population_inicialization(population_size, min_carac, max_carac)

In [43]:
# Treino do primeiro modelo
gen, gen_avg, gen_max_values, gen_min_values, generation_win_rate, gen_max_ind = treino(population_1, maps, max_generation, stedy_rate, mutation_rate, crossover_rate)

Generation 0
Avg = -248.13666666666668 
Win rate = 0.0
Max Value = 31.333333333333332
Min value = -429.6666666666667
max_ind = (array([47], dtype=int64),)

Generation 1
Avg = -204.56666666666663 
Win rate = 0.0
Max Value = 190.0
Min value = -420.3333333333333
max_ind = (array([31], dtype=int64),)

Generation 2
Avg = -160.09090909090907 
Win rate = 0.0
Max Value = 190.0
Min value = -404.6666666666667
max_ind = (array([0], dtype=int64),)

Generation 3
Avg = -132.9462365591398 
Win rate = 0.0033333333333333335
Max Value = 373.6666666666667
Min value = -397.3333333333333
max_ind = (array([71], dtype=int64),)

Generation 4
Avg = -145.84782608695653 
Win rate = 0.0033333333333333335
Max Value = 373.6666666666667
Min value = -386.3333333333333
max_ind = (array([0], dtype=int64),)

Generation 5
Avg = -129.16287878787878 
Win rate = 0.0033333333333333335
Max Value = 373.6666666666667
Min value = -438.0
max_ind = (array([0], dtype=int64),)

Generation 6
Avg = -105.7921568627451 
Win rate = 0.006

TypeError: can only concatenate list (not "int") to list

In [ ]:
## Parâmetros do segundo modelo
population_size = 50
max_generation = 1000
stedy_rate = 0.05
mutation_rate = 0.15
crossover_rate = 0.8
min_carac = 15
max_carac = 30
population_2 = population_inicialization(population_size, min_carac, max_carac)

In [ ]:
#  Minor changes and second test, still repeats the individuals
gen_2, gen_avg_2, gen_max_values_2, gen_min_values_2, generation_win_rate_2, gen_max_ind_2 = treino(population_2, maps, max_generation, stedy_rate, mutation_rate, crossover_rate)

In [ ]:
## Parâmetros do terceiro modelo
population_size = 75
max_generation = 1000
stedy_rate = 0.05
mutation_rate = 0.2
crossover_rate = 0.75
min_carac = 25
max_carac = 35
population_3 = population_inicialization(population_size, min_carac, max_carac)

In [ ]:
# changes the ratio to 0.5% and the selection of repetead individuals
gen_3, gen_avg_3, gen_max_values_3, gen_min_values_3, generation_win_rate_3, gen_max_ind_3 = treino(population_3, maps, max_generation, stedy_rate, mutation_rate, crossover_rate) 

In [ ]:
inds = ['RRLRLLRLLRRLLLLRLRLLRLRLLRRRRRRLLRR']
for i in inds:
    for mapa in maps:
        result = !python pacman.py -p EvolutionAgent -a steps=$i -l $mapa -f

# Reinforcement Learning:
* The MDP formulation (states, actions, reward function)
* The discretization model adopted
* The number of training episodes
* The stop criteria
* The learning rate value and other parameter values used

# Comparative